In [1]:
import os, re, pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import BigInteger, Integer, Numeric, Text, Date, Boolean

In [2]:
engine = create_engine(os.getenv("DATABASE_URL"))

In [3]:
listings = pd.read_sql("SELECT * FROM raw.listings", engine)
reviews = pd.read_sql("SELECT * FROM raw.reviews", engine)

In [4]:
listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,264776,https://www.airbnb.com/rooms/264776,20250610032232,2025-06-11,city scrape,Huge Four Bedroom Apartment,An extremely large and sunny four bedroom grou...,Catford is a well established London suburb. T...,https://a0.muscache.com/pictures/hosting/Hosti...,1389063,...,4.74,4.62,4.72,None,t,11,11,0,0,0.51
1,264777,https://www.airbnb.com/rooms/264777,20250610032232,2025-06-11,city scrape,One Bedroom Apartment,Recently refurbished sunny one bedroom first f...,None,https://a0.muscache.com/pictures/hosting/Hosti...,1389063,...,4.25,4.54,4.42,None,t,11,11,0,0,0.22
2,264778,https://www.airbnb.com/rooms/264778,20250610032232,2025-06-11,city scrape,Two Bedroom Newly Refurbished Apartment,A large and sunny two bedroom second floor apa...,Catford is a well established London suburb. T...,https://a0.muscache.com/pictures/50662093/af12...,1389063,...,4.52,4.36,4.38,None,t,11,11,0,0,0.43
3,264779,https://www.airbnb.com/rooms/264779,20250610032232,2025-06-11,city scrape,Refurbished Two Bedroom Apartment,A large and sunny two bedroom second floor apa...,Catford is a well established London suburb. T...,https://a0.muscache.com/pictures/50660860/e440...,1389063,...,4.61,4.5,4.47,None,t,11,11,0,0,0.30
4,264780,https://www.airbnb.com/rooms/264780,20250610032232,2025-06-11,city scrape,Spacious refurbished 2 bedroom apt with balcony,Completely refurbished 2 bedroom apt to sleep ...,None,https://a0.muscache.com/pictures/airflow/Hosti...,1389063,...,4.74,4.37,4.59,None,t,11,11,0,0,0.35


In [5]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96651 entries, 0 to 96650
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   id                                            96651 non-null  object
 1   listing_url                                   96651 non-null  object
 2   scrape_id                                     96651 non-null  object
 3   last_scraped                                  96651 non-null  object
 4   source                                        96651 non-null  object
 5   name                                          96651 non-null  object
 6   description                                   93806 non-null  object
 7   neighborhood_overview                         41984 non-null  object
 8   picture_url                                   96642 non-null  object
 9   host_id                                       96651 non-null  object
 10

In [6]:
listings['price'] = (listings['price'].str.replace(r'[^0-9.]', '', regex=True).astype(float))

In [7]:
listings['price']

0         297.0
1          98.0
2         148.0
3         144.0
4         157.0
          ...  
96646    1395.0
96647      95.0
96648     299.0
96649    3000.0
96650     132.0
Name: price, Length: 96651, dtype: float64

In [8]:
listings.isnull().values.any()

np.True_

In [9]:
#Filtering all columns with NaNs
listings.isnull().sum()[listings.isnull().sum() > 0]

description                      2845
neighborhood_overview           54667
picture_url                         9
host_name                          38
host_since                         38
host_location                   23328
host_about                         38
host_response_time                 38
host_response_rate                 38
host_acceptance_rate               38
host_is_superhost                1823
host_thumbnail_url                 38
host_picture_url                   38
host_neighbourhood              50360
host_listings_count                38
host_total_listings_count          38
host_has_profile_pic               38
host_identity_verified             38
neighbourhood                   54667
neighbourhood_group_cleansed    96651
bathrooms                       33921
bathrooms_text                    149
bedrooms                        12580
beds                            33975
price                           33967
calendar_updated                96651
has_availabi

In [10]:
#Filtering the price column to check the associated null values
listings[listings['price'].isnull()][['price']]

,price
12,NaN
13,NaN
14,NaN
17,NaN
30,NaN
...,...
93358,NaN
93377,NaN
93394,NaN
93424,NaN


In [11]:
#Replace the missing price values with the series mean
listings['price'] = listings['price'].fillna(listings['price'].mean())

In [12]:
#checking if the price column still has nulls
listings['price'].isnull().values.any()

np.False_

In [13]:
#removing columns with more the 50% missing values
listings = listings.drop(columns=['neighbourhood_group_cleansed','calendar_updated','license'])

In [14]:
#Filtering all columns with NaNs
listings.isnull().sum()[listings.isnull().sum() > 0]

description                     2845
neighborhood_overview          54667
picture_url                        9
host_name                         38
host_since                        38
host_location                  23328
host_about                        38
host_response_time                38
host_response_rate                38
host_acceptance_rate              38
host_is_superhost               1823
host_thumbnail_url                38
host_picture_url                  38
host_neighbourhood             50360
host_listings_count               38
host_total_listings_count         38
host_has_profile_pic              38
host_identity_verified            38
neighbourhood                  54667
bathrooms                      33921
bathrooms_text                   149
bedrooms                       12580
beds                           33975
has_availability                4463
estimated_revenue_l365d        33967
first_review                   25164
last_review                    25164
r

In [15]:
listings['host_identity_verified']

0        t
1        t
2        t
3        t
4        t
        ..
96646    t
96647    t
96648    t
96649    t
96650    t
Name: host_identity_verified, Length: 96651, dtype: object

In [16]:
listings['host_since'] = pd.to_datetime(listings['host_since'])

In [17]:
print(listings['host_since'].dtype)

datetime64[ns]


In [18]:
listings['host_since'] = listings['host_since'].ffill()

In [19]:
#checking if the host_since column still has nulls
listings['host_since'].isnull().values.any()

np.False_

In [20]:
listings['host_response_rate']

0         86%
1         86%
2         86%
3         86%
4         86%
         ... 
96646     96%
96647    100%
96648     90%
96649     99%
96650    100%
Name: host_response_rate, Length: 96651, dtype: object

In [21]:
listings['host_response_rate'] = pd.to_numeric(listings['host_response_rate'].str.rstrip('%'),errors='coerce')/100

In [22]:
#Replace the missing host response rate values with the series mean
listings['host_response_rate'] = listings['host_response_rate'].fillna(listings['host_response_rate'].mean())

In [23]:
#checking if the column still has nulls
listings['host_response_rate'].isnull().values.any()

np.False_

In [24]:
listings['host_acceptance_rate'] = pd.to_numeric(listings['host_acceptance_rate'].str.rstrip('%'),errors='coerce')/100

In [25]:
#Replace the missing host response rate values with the series mean
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].fillna(listings['host_acceptance_rate'].mean())

In [26]:
#checking if the column still has nulls
listings['host_response_rate'].isnull().values.any()

np.False_

In [27]:
#converting column into number column
listings['host_listings_count'] = pd.to_numeric(listings['host_listings_count'],errors='coerce')

In [28]:
#Replace the missing host listings count values with 1
listings['host_listings_count'] = listings['host_listings_count'].fillna(1)

In [29]:
#checking if the column still has nulls
listings['host_response_rate'].isnull().values.any()

np.False_

In [30]:
#converting column into number column
listings['host_total_listings_count'] = pd.to_numeric(listings['host_total_listings_count'],errors='coerce')
#Replace the missing host listings count values with 1
listings['host_total_listings_count'] = listings['host_total_listings_count'].fillna(1)

In [31]:
#checking if the column still has nulls
listings['host_total_listings_count'].isnull().values.any()

np.False_

In [32]:
#Filtering all columns with NaNs
listings.isnull().sum()[listings.isnull().sum() > 0]

description                     2845
neighborhood_overview          54667
picture_url                        9
host_name                         38
host_location                  23328
host_about                        38
host_response_time                38
host_is_superhost               1823
host_thumbnail_url                38
host_picture_url                  38
host_neighbourhood             50360
host_has_profile_pic              38
host_identity_verified            38
neighbourhood                  54667
bathrooms                      33921
bathrooms_text                   149
bedrooms                       12580
beds                           33975
has_availability                4463
estimated_revenue_l365d        33967
first_review                   25164
last_review                    25164
review_scores_rating           25164
review_scores_accuracy         25179
review_scores_cleanliness      25173
review_scores_checkin          25208
review_scores_communication    25185
r

In [33]:
listings = listings.drop(columns=['neighborhood_overview','host_neighbourhood','neighbourhood'])

In [34]:
#Filtering all columns with NaNs
listings.isnull().sum()[listings.isnull().sum() > 0]

description                     2845
picture_url                        9
host_name                         38
host_location                  23328
host_about                        38
host_response_time                38
host_is_superhost               1823
host_thumbnail_url                38
host_picture_url                  38
host_has_profile_pic              38
host_identity_verified            38
bathrooms                      33921
bathrooms_text                   149
bedrooms                       12580
beds                           33975
has_availability                4463
estimated_revenue_l365d        33967
first_review                   25164
last_review                    25164
review_scores_rating           25164
review_scores_accuracy         25179
review_scores_cleanliness      25173
review_scores_checkin          25208
review_scores_communication    25185
review_scores_location         25209
review_scores_value            25209
reviews_per_month              25164
d

In [35]:
#convert bathrooms_text into number column
listings['bathrooms_text'] = pd.to_numeric(listings['bathrooms_text'].str.replace(r'\s*baths?\b', '', regex=True), errors='coerce')

In [36]:
#Replace the missing bathroom values with 0
listings['bathrooms_text'] = listings['bathrooms_text'].fillna(0)

In [37]:
#checking if the column still has nulls
listings['host_response_rate'].isnull().values.any()

np.False_

In [38]:
def impute_bedrooms_and_beds(df):
    """
    Imputes missing values in 'bedrooms' and 'beds' using each other.
    Assumes 1 bed per bedroom and vice versa if one is missing.
    """
    # Ensure numeric types
    df['bedrooms'] = pd.to_numeric(df['bedrooms'], errors='coerce')
    df['beds'] = pd.to_numeric(df['beds'], errors='coerce')

    # Impute beds using bedrooms
    missing_beds = df['beds'].isnull() & df['bedrooms'].notnull()
    df.loc[missing_beds, 'beds'] = df.loc[missing_beds, 'bedrooms']

    # Impute bedrooms using beds
    missing_bedrooms = df['bedrooms'].isnull() & df['beds'].notnull()
    df.loc[missing_bedrooms, 'bedrooms'] = df.loc[missing_bedrooms, 'beds']
    
    return df

In [39]:
listings = impute_bedrooms_and_beds(listings)

In [40]:
def impute_missing_values(df,col,fill_value=0):
    #Replace the missing values with the fill value
    df[col] = df[col].fillna(fill_value)
    return df

In [41]:
for col in ['beds','bedrooms']:
    listings = impute_missing_values(listings,col,fill_value=1)

In [42]:
#checking if the column still has nulls
listings['bedrooms'].isnull().values.any()

np.False_

In [43]:
for col in ['host_has_profile_pic','host_identity_verified','has_availability']:
    listings = impute_missing_values(listings,col,fill_value='f')

In [44]:
#checking if the column still has nulls
listings['has_availability'].isnull().values.any()

np.False_

In [45]:
def convert_to_num_and_impute(df,col):
    #converting column into number column
    df[col] = pd.to_numeric(df[col],errors='coerce')
    #calculate mean value
    mean_value = df[col].mean()
    #Replace the missing values with fill_value
    df[col] = df[col].fillna(mean_value)
    return df

In [46]:
review_columns = ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                  'review_scores_checkin','review_scores_communication','review_scores_location',
                  'review_scores_value','reviews_per_month']
for col in review_columns:
    listings = convert_to_num_and_impute(listings,col)

In [47]:
listings['review_scores_communication'].isnull().values.any()

np.False_

In [48]:
def convert_to_datetime_and_impute(df,col):
    #convert columns to date_time
    df[col] = pd.to_datetime(df[col])
    #impute missing date with previous date 
    df[col] = df[col].ffill()
    return df

In [49]:
for col in ['first_review','last_review']:
    listings = convert_to_datetime_and_impute(listings,col)

In [50]:
#Drop the bathrooms the column. its similar to bathroom_text
listings = listings.drop(columns='bathrooms')

In [51]:
#Filtering all columns with NaNs
listings.isnull().sum()[listings.isnull().sum() > 0]

description                 2845
picture_url                    9
host_name                     38
host_location              23328
host_about                    38
host_response_time            38
host_is_superhost           1823
host_thumbnail_url            38
host_picture_url              38
estimated_revenue_l365d    33967
dtype: int64

In [52]:
listings['estimated_revenue_l365d']

0        32670
1         3626
2         8140
3         9216
4         5809
         ...  
96646        0
96647        0
96648        0
96649        0
96650        0
Name: estimated_revenue_l365d, Length: 96651, dtype: object

In [53]:
listings = impute_missing_values(listings,'host_is_superhost',fill_value='f')

In [54]:
#converting column into number column
listings['estimated_revenue_l365d'] = pd.to_numeric(listings['estimated_revenue_l365d'],errors='coerce')
#Replace the missing values with fill_value
listings['estimated_revenue_l365d'] = listings['estimated_revenue_l365d'].fillna(0)

In [55]:
#Filtering all columns with NaNs
listings.isnull().sum()[listings.isnull().sum() > 0]

description            2845
picture_url               9
host_name                38
host_location         23328
host_about               38
host_response_time       38
host_thumbnail_url       38
host_picture_url         38
dtype: int64

In [56]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96651 entries, 0 to 96650
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            96651 non-null  object        
 1   listing_url                                   96651 non-null  object        
 2   scrape_id                                     96651 non-null  object        
 3   last_scraped                                  96651 non-null  object        
 4   source                                        96651 non-null  object        
 5   name                                          96651 non-null  object        
 6   description                                   93806 non-null  object        
 7   picture_url                                   96642 non-null  object        
 8   host_id                                       96651 non-null  obje

In [57]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'picture_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_

In [58]:
listings.head()[['minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365','calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms']]

,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,3,365,3,3,1125,1125,3.0,1125.0,13,15,31,293,11,11,0,0
1,3,365,3,3,1125,1125,3.0,1125.0,0,13,43,318,11,11,0,0
2,3,365,1,3,1125,1125,3.0,1125.0,3,6,27,302,11,11,0,0
3,3,365,3,3,1125,1125,3.0,1125.0,11,33,53,328,11,11,0,0
4,3,365,3,3,1125,1125,3.0,1125.0,0,9,11,255,11,11,0,0


In [59]:
num_columns = ['minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365','calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms']
for col in num_columns:
    listings = convert_to_num_and_impute(listings,col)

In [60]:
listings.head()[['number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy',
       'number_of_reviews_ly', 'estimated_occupancy_l365d']]

,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d
0,68,12,1,134,15,110
1,24,4,0,157,7,37
2,58,6,0,141,8,55
3,36,7,0,167,7,64
4,54,4,0,101,7,37


In [61]:
nm_columns = ['number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy',
       'number_of_reviews_ly', 'estimated_occupancy_l365d']
for col in nm_columns:
    listings = convert_to_num_and_impute(listings,col)

In [62]:
listings.head()[['latitude', 'longitude', 'property_type',
       'room_type', 'accommodates','amenities','host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified']]

,latitude,longitude,property_type,room_type,accommodates,amenities,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,51.44306,-0.01948,Entire rental unit,Entire home/apt,10,"[""Cooking basics"", ""Washer"", ""Iron"", ""Smoke al...",11.0,12.0,"['email', 'phone']",t,t
1,51.44284,-0.01997,Entire rental unit,Entire home/apt,2,"[""Cooking basics"", ""Wine glasses"", ""Iron"", ""Sm...",11.0,12.0,"['email', 'phone']",t,t
2,51.44359,-0.02275,Entire rental unit,Entire home/apt,4,"[""Cooking basics"", ""Iron"", ""Smoke alarm"", ""Fir...",11.0,12.0,"['email', 'phone']",t,t
3,51.44355,-0.02309,Entire rental unit,Entire home/apt,5,"[""Cooking basics"", ""Iron"", ""Smoke alarm"", ""Fir...",11.0,12.0,"['email', 'phone']",t,t
4,51.44333,-0.02307,Entire rental unit,Entire home/apt,4,"[""Cooking basics"", ""Washer"", ""Iron"", ""Smoke al...",11.0,12.0,"['email', 'phone']",t,t


In [63]:
nt_colms = ['latitude', 'longitude', 'accommodates','host_listings_count',
       'host_total_listings_count']
for col in nt_colms:
    listings = convert_to_num_and_impute(listings,col)

In [64]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96651 entries, 0 to 96650
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            96651 non-null  object        
 1   listing_url                                   96651 non-null  object        
 2   scrape_id                                     96651 non-null  object        
 3   last_scraped                                  96651 non-null  object        
 4   source                                        96651 non-null  object        
 5   name                                          96651 non-null  object        
 6   description                                   93806 non-null  object        
 7   picture_url                                   96642 non-null  object        
 8   host_id                                       96651 non-null  obje

In [65]:
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from scipy import sparse

In [66]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'picture_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_

In [ ]:
['id', 'name','description',  'host_id', 'host_name',
       'host_since', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds','price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy',
       'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

In [97]:
listings['estimated_occupancy_l365d']

0        110
1         37
2         55
3         64
4         37
        ... 
96646      0
96647      0
96648      0
96649      0
96650      0
Name: estimated_occupancy_l365d, Length: 96651, dtype: int64

In [98]:
columns_to_keep = ['id', 'name','description',  'host_id', 'host_name',
       'host_since', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds','price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'availability_eoy',
       'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

df_base = listings[columns_to_keep].copy()

df_base.to_sql(
    "listings_base",
    engine,
    schema="cleaned",
    if_exists="replace",
    index=False,
    chunksize=5000  # small, safe batches
)

8409

In [100]:
len(df_base)

96651

In [101]:
clean_cnt = pd.read_sql("SELECT COUNT(*) FROM cleaned.listings_base", engine).iloc[0,0]
clean_cnt

np.int64(96651)

In [102]:
# Parse 'amenities' safely into Python lists
def parse_amenities(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):  # NaN
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        s = x.replace("{","[").replace("}","]")  # some dumps use {}
        try:
            val = ast.literal_eval(s)
            return val if isinstance(val, (list, tuple, set)) else []
        except Exception:
            return []
    return []


In [103]:
amen_lists = listings["amenities"].apply(parse_amenities)

# keep only frequent amenities to avoid 1000s of columns
from collections import Counter
cnt = Counter(a.strip().lower() for lst in amen_lists for a in set(lst))
MIN_COUNT = 300  # tweak (e.g., 200–500). Lower = more columns
kept = {a for a,c in cnt.items() if c >= MIN_COUNT}
amen_lists = amen_lists.apply(lambda lst: [str(a).strip().lower() for a in lst if str(a).strip().lower() in kept])

# One-hot encode as **sparse**
mlb = MultiLabelBinarizer(sparse_output=True)  # <-- key difference
X_sparse = mlb.fit_transform(amen_lists)       # scipy CSR matrix

# Convert to a pandas **sparse** DataFrame to save RAM
safe_col = lambda s: "amen__" + s.replace(" ", "_").replace("/", "_")
amen_cols = [safe_col(c) for c in mlb.classes_]
amen_df = pd.DataFrame.sparse.from_spmatrix(X_sparse, index=listings.index, columns=amen_cols)

# smaller dtype
for c in amen_df.columns:
    amen_df[c] = amen_df[c].astype(pd.SparseDtype("int8", fill_value=0))

In [105]:
amen_df.columns

Index(['amen__air_conditioning', 'amen__arcade_games', 'amen__baby_bath',
       'amen__baby_bath_-_available_upon_request', 'amen__baby_monitor',
       'amen__baby_safety_gates', 'amen__babysitter_recommendations',
       'amen__backyard', 'amen__baking_sheet', 'amen__barbecue_utensils',
       ...
       'amen__toaster', 'amen__trash_compactor', 'amen__tv',
       'amen__tv_with_standard_cable', 'amen__washer',
       'amen__washer_– in_unit', 'amen__waterfront', 'amen__wifi',
       'amen__window_guards', 'amen__wine_glasses'],
      dtype='object', length=207)

In [106]:
amen_df.to_sql(
    "listings_amenities_wide",
    engine,
    schema="cleaned",
    if_exists="replace",
    index=False,
    chunksize=5000
)

2608